In [1]:
import os.path
import numpy as np
import matplotlib.pyplot as plt
import scipy
import refnx
from refnx.dataset import ReflectDataset, Data1D
from refnx.analysis import Transform, CurveFitter, Objective, Model, Parameter
from refnx.reflect import SLD, Slab, ReflectModel
print('refnx: %s\nscipy: %s\nnumpy: %s' % (refnx.version.version,
                   scipy.version.version, np.version.version))


refnx: 0.1.9
scipy: 1.3.0
numpy: 1.16.4


In [2]:
# simple structure
thicks_true = [10,120,100,130,10]
bs_true = [0,5,4.5,5,10]
roughs_true = [0,0,0,0,0]
# bs = [5,4.5,5]
# thicks = [120,100,130]
# roughs = [0,0,0]
bs = bs_true
thicks = thicks_true
roughs = roughs_true
loss = 1.
no_layers = len(bs)
no_layers = len(bs_true)
# no_layers = 3
layers = []
names = []
#names = ['layer1','layer2']
for i in range(no_layers):
    names.append('layer'+str(i))
for i in range(no_layers):
        sld = SLD(bs[i],name=names[i])
        layers.append(sld(thicks[i],roughs[i]))
structure = layers[0]
for layer in layers[1:]:
    structure|=layer
print("done")

done


In [3]:
print(structure)

________________________________________________________________________________
Structure:                
solvent: None
reverse structure: False
contract: 0

________________________________________________________________________________
Parameters:    'layer0'    
<Parameter:'layer0 - thick', value=10 (fixed)  , bounds=[-inf, inf]>
<Parameter:'layer0 - sld' , value=0 (fixed)  , bounds=[-inf, inf]>
<Parameter:'layer0 - isld', value=0 (fixed)  , bounds=[-inf, inf]>
<Parameter:'layer0 - rough', value=0 (fixed)  , bounds=[-inf, inf]>
<Parameter:'layer0 - volfrac solvent', value=0 (fixed)  , bounds=[-inf, inf]>
________________________________________________________________________________
Parameters:    'layer1'    
<Parameter:'layer1 - thick', value=120 (fixed)  , bounds=[-inf, inf]>
<Parameter:'layer1 - sld' , value=5 (fixed)  , bounds=[-inf, inf]>
<Parameter:'layer1 - isld', value=0 (fixed)  , bounds=[-inf, inf]>
<Parameter:'layer1 - rough', value=0 (fixed)  , bounds=[-inf, inf]>
<

In [4]:
%matplotlib notebook
plt.plot(*structure.sld_profile())
plt.ylabel('SLD /$10^{-6} \AA^{-2}$')
plt.xlabel('distance / $\AA$');
print("done")

<IPython.core.display.Javascript object>

done


In [5]:
#dq/q = resolution = 5 == 5%
resolution = 5.
model = ReflectModel(structure, bkg=3e-6, dq=resolution)
# model.scale.setp(bounds=(0.6, 1.2), vary=True)
# model.bkg.setp(bounds=(1e-9, 9e-6), vary=True)
print("done")

done


In [ ]:
#%matplotlib notebook
q = np.linspace(0.005, 0.3, 1001)
plt.figure(2)
plt.plot(q, model(q, x_err=resolution))
plt.xlabel('Q')
plt.ylabel('Reflectivity')
plt.yscale('log')
#Q, R(Q), sig_Q, sig_R =
R = model(q, x_err=resolution)
import make_sim_data as msd
sim_dR = msd.main(sim_q=q,sim_R=R)
R = model(q, x_err=resolution)

# q = np.linspace(0.005, 0.3, 1001)
# plt.figure(3)
# plt.plot(q, model(q, x_err=resolution))
# plt.xlabel('Q')
# plt.ylabel('Reflectivity')
# plt.yscale('log')
plt.errorbar(q,R,sim_dR, alpha=0.4)
#point to introduce loss

In [8]:
import nLayerModel as nlm
# simple structure
maxThickness = 350
# structs = []
# fitrs = []
# ln_posts = []
#%matplotlib notebook
fig_i = 3
print(q, R, sim_dR)
data = Data1D(data=(q, R, sim_dR))
nLayers = len(thicks_true)

air = SLD(0,"air")
airSlab = air(10,0)
airSlab.thick.setp(vary=True, bounds=(1,30))

sio2 = SLD(10,"sio2")
sio2Slab = sio2(10,0)
sio2Slab.thick.setp(vary=True, bounds=(1,30))

bsIn = []
thicksIn = []
roughsIn = []

name = "true"
bsIn = [5,5,5,5,5]
thicksIn = [maxThickness/nLayers for i in range(nLayers)]
roughsIn = [0,0,0,0,0]
for i in range(nLayers):
    b = Parameter(bs[i], "b %d" % i,vary=True, bounds=(0,10))#bs[i]-0.5, bs[i]+0.5))
    bsIn.append(b)

    thick = Parameter(thicks[i], "thick %d" % i,vary=True, bounds=(50,200))#thicks[i]-2, thicks[i]+2))
    thicksIn.append(thick)

    if roughs[i]-1 <= 0:
        r = 0
    else:
        r = roughs[i]-1
    rough = Parameter(roughs[i], "rough %d" % i,vary=True, bounds=(0,10))#r, roughs[i]+1))
    roughsIn.append(rough)

nlmodel = nlm.NLayerModel("nlayered", bsIn, thicksIn, roughsIn, maxThickness)

nlmodel = airSlab|nlmodel|sio2Slab
model = ReflectModel(nlmodel)
objective = Objective(model, data)
fitter = CurveFitter(objective)
fitter.fit('differential_evolution');
plt.figure(3)
global_objective.plot()
plt.yscale('log')
plt.xlabel('Q')
plt.ylabel('Reflectivity')
plt.legend();

[0.005    0.005295 0.00559  ... 0.29941  0.299705 0.3     ] [9.99959417e-01 9.99959417e-01 9.99959417e-01 ... 3.91691449e-06
 3.91334180e-06 3.91063741e-06] [2.15664855e-03 2.03649532e-03 1.92902374e-03 ... 2.19007143e-07
 2.18781005e-07 2.18557877e-07]


LinAlgError: The following Parameters have no effect on Objective.residuals, please consider fixing them.
[Parameter(value=24.40983329184865, name='air - thick', vary=True, bounds=Interval(lb=1, ub=30), constraint=None), Parameter(value=0.0, name='rough 3', vary=True, bounds=Interval(lb=0, ub=10), constraint=None), Parameter(value=14.833326442483388, name='sio2 - thick', vary=True, bounds=Interval(lb=1, ub=30), constraint=None)]